In [1]:
import sys

sys.path.append("megai_man")

from hyperparam_optimization import tune
import optuna

In [2]:
def sample_params(trial: optuna.Trial, n_envs: int, n_steps: int):
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    clip_range = trial.suggest_float("clip_range", 0.1, 0.2, step=0.1)
    vf_coef = trial.suggest_float("vf_coef", 0.0, 1.0)
    ent_coef = trial.suggest_float("ent_coef", 1e-5, 1e-1, log=True)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0, log=True)
    n_epochs = trial.suggest_int("n_epochs", 1, 10)
    gamma = trial.suggest_categorical("gamma", [0.99, 0.995, 0.999])
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 1)
    features_dim_exp = trial.suggest_int("features_dim_power", 8, 10)
    features_dim = 2**features_dim_exp

    n_steps = 256
    n_envs = 8
    return {
        "n_steps": n_steps,
        "batch_size": n_envs * n_steps,
        "learning_rate": learning_rate,
        "clip_range": clip_range,
        "vf_coef": vf_coef,
        "ent_coef": ent_coef,
        "gae_lambda": gae_lambda,
        "n_epochs": n_epochs,
        "gamma": gamma,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "features_extractor_kwargs": {
                "features_dim": features_dim,
            },
        },
    }


tune(
    sample_fn=sample_params,
    name="cutman_random_searcher_nsteps256",
    n_trials=100,
    timesteps_per_trial=1_000_000,
)

[I 2024-01-26 15:13:50,066] A new study created in RDB with name: cutman_random_searcher_nsteps256


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-01-26 15:29:06,910] Trial 0 finished with value: 1.0 and parameters: {'learning_rate': 0.0006335027251434691, 'clip_range': 0.2, 'vf_coef': 0.6765143359376254, 'ent_coef': 0.008155155172198747, 'gae_lambda': 0.9892266094139247, 'n_epochs': 1, 'gamma': 0.99, 'max_grad_norm': 0.6556464140369045, 'features_dim_power': 8}. Best is trial 0 with value: 1.0.
[I 2024-01-26 15:44:18,885] Trial 1 finished with value: 3.0 and parameters: {'learning_rate': 0.000947581722600463, 'clip_range': 0.1, 'vf_coef': 0.7008447521659813, 'ent_coef': 0.00014890607747597575, 'gae_lambda': 0.950921793896334, 'n_epochs': 1, 'gamma': 0.999, 'max_grad_norm': 0.31626540946723275, 'features_dim_power': 10}. Best is trial 1 with value: 3.0.
[I 2024-01-26 16:01:18,314] Trial 2 finished with value: 3.0 and parameters: {'learning_rate': 2.2916050351570544e-05, 'clip_range': 0.1, 'vf_coef': 0.9091795928395876, 'ent_coef': 0.08191962207169494, 'gae_lambda': 0.9009981276344472, 'n_epochs': 3, 'gamma': 0.999, 'max_g

KeyboardInterrupt: 

how the fuck Trial 7 reached average final distance of >500 if max distance in the map is 498?

Trial 67: bs2048_lr4.29e-04_cr0.2_vc3.48e-01_ec4.36e-03_gl8.60e-01_ne7_g0.995_mgn9.00e-01_fd512